In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
news = pd.read_csv('../dataset_processed/fakeNews_titleStemming_Sampled.csv', index_col=0)
news.reset_index(inplace=True)
news_predictors = news.drop(columns = ['label'])
news_target = news['label']

news_predictors

,level_0,000,00pm,10,100,1000,100percentfedup,11,12,12000,...,zee,zeleny,zero,ziff,zika,zionist,zone,zoo,zuckerberg,title
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,how (and why) to camp in the nation park - the...
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,how much is donald trump hurt the g.o.p.? here...
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,huffington post continu campaign for hillari c...
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,dear mainstream media: you made deepstateg hap...
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"for those who have it all, charit wed registri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,proof hillari clinton and her foundat are work...
996,996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,shia labeouf shout at wall to kick off 4-year-...
997,997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,comment on texa polic arrest trump voter for w...
998,998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train french soldier to supervis daesh


In [15]:
## Setup knn classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

parms_knn = {
         'n_neighbors': [10, 20, 30, 100],
         'weights': ['uniform','distance'],
         'p': [1,2]
        }

grid = GridSearchCV(
    KNeighborsClassifier(), parms_knn, cv = 5, scoring = 'accuracy').fit(news_predictors, news_target)
print(grid.best_estimator_)
grid.best_score_


KNeighborsClassifier(n_neighbors=30, p=1)


0.6100000000000001

# RNN

In [ ]:
import torch
import torchvision

In [ ]:
from torch import nn
class news_rnn(nn.Module):
    
    ## Constructor commands
    def __init__(self, input_size, hidden_size, output_size):
        super(news_rnn, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    ## Function to generate predictions
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [ ]:
## Initialize model with random weights
input_dim = news_predictors.shape[1]
rnn = news_rnn(input_dim, 100, 2)

# Random Forest 

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Create list of models in the ensemble
base_models = [('logreg', LogisticRegression(penalty='none')),
              ('tree', DecisionTreeClassifier(max_depth=5)),
              ('knn', KNeighborsClassifier(n_neighbors=20))]

## Final model used to aggregate predicted probs
my_final_estimator = DecisionTreeClassifier(max_depth=3)

## Create stack
from sklearn.ensemble import StackingClassifier
my_stack = StackingClassifier(estimators = base_models, final_estimator = my_final_estimator, 
                              stack_method ='predict_proba', cv=5)

## Fit and Evaluate (cv is done internally in StackingClassifier)
fitted_stack = my_stack.fit(news_predictors, news_target)
cv_stacked_preds = fitted_stack.predict(news_predictors)

from sklearn.metrics import accuracy_score

accuracy_score(news_target, cv_stacked_preds)




0.9566666666666667